In [54]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import absolute
import math
from numpy import arange

from pandas import read_csv
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
import seaborn as sns
import pylab

from scipy import stats
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV


In [55]:
#THE CCT DATA
dataframe = pd.read_csv('CCT-TFS syyskuu Henry.csv')
dataframe
print(dataframe.shape)
#print(dataframe.head(n=4))
dataframe
dataframe.dropna(inplace=True)

(1272, 14)


In [250]:
#CENTERING THE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#centered_dat=boston_features_df.copy()
new_dat=dataframe.drop(['Steel'], axis=1)
centered_dat=pd.DataFrame(scaler.fit_transform(new_dat), columns=new_dat.columns)
#centered_dat.head()



,C,Si,Mn,Cr,Mo,Ni,Al,Cu,ppmB,Pa,Rc,lnRc
0,0.130952,0.102662,0.043738,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.316325,0.387089,0.915697
1,0.130952,0.102662,0.043738,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.316325,0.054827,0.742108
2,0.130952,0.102662,0.043738,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.316325,0.004181,0.513751
3,0.130952,0.102662,0.041750,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.316325,0.000310,0.285918
4,0.130952,0.102662,0.041750,0.014599,0.000000,0.011342,0.000000,0.108374,0.000000,0.316325,0.000013,0.061569
...,...,...,...,...,...,...,...,...,...,...,...,...
1267,0.214286,0.136882,0.159046,0.204380,0.007576,0.003781,0.023697,0.004926,0.886076,0.669214,0.038697,0.711169
1268,0.214286,0.136882,0.159046,0.204380,0.007576,0.003781,0.023697,0.004926,0.886076,0.669214,0.019342,0.649600
1269,0.214286,0.136882,0.159046,0.204380,0.007576,0.003781,0.023697,0.004926,0.886076,0.669214,0.009665,0.588031
1270,0.214286,0.152091,0.353877,0.049270,0.007576,0.217391,0.023697,0.004926,0.886076,0.669214,0.009665,0.588031


In [57]:
#NORMALIZING THE DATA
#dat=dataframe.drop(['Steel'],axis=1)
scaler=MinMaxScaler()
new_dat[new_dat.columns.values] = scaler.fit_transform(new_dat)
#new_dat

#STANDARDIZED DATA; CENTERED(WITHOUT MEAN) + NORMALIZED (KEPT WITHIN A RANGE)

In [58]:
#DEPENDENT AND INDEPENDENT VARIABLES
#x=Dependent Variables
#y=Independent Variables

data = pd.DataFrame.values
x=new_dat.drop(['Texp'], axis = 1)
y=new_dat.Texp
x=x.astype(float)
y=y.astype(float)


In [78]:
data = pd.DataFrame.values

x=new_dat.drop(['Texp'], axis = 1)
y=new_dat[['Texp']]
x=x.astype(float)
y=y.astype(float)
#model = sm.OLS(y, sm.add_constant(x)).fit()

new_x=sm.add_constant(x)
trans_x=new_x.T


In [79]:
# FINDING THE RESIDUALS FROM THE ORDINARY LEAST SQUARES
model_Simple = sm.OLS(y, x).fit()

model = sm.OLS(y, sm.add_constant(x)).fit()

residuals=model.resid
#residuals


In [204]:
#CALCULATING THE BETA FOR GLS
data = pd.DataFrame.values

x=new_dat.drop(['Texp'], axis = 1)
y=new_dat[['Texp']]
x=x.astype(float)
y=y.astype(float)
#model = sm.OLS(y, sm.add_constant(x)).fit()

new_x=sm.add_constant(x)
trans_x=new_x.T

residuals = pd.DataFrame(data=residuals)
residuals
#FINDING THE COVARIANCE OF THE RESIDUALS FROM OLS, i.e Ohms
from scipy.linalg import toeplitz
#ohm=ohm.to_numpy()

ohm = toeplitz(residuals)
cc=np.fill_diagonal(ohm, 0)

#ohm = np.cov(,bias=True)
ohm=pd.DataFrame(data=ohm)

ohm_inv = np.linalg.inv(ohm)
trans_x = pd.DataFrame(data=trans_x)

x_trans_by_ohm_inv = trans_x.dot(ohm_inv)
x_trans_by_ohm_inv_by_x = x_trans_by_ohm_inv.dot(new_x)

x_trans_by_ohm_inv_by_y = x_trans_by_ohm_inv.dot(y)
x_trans_by_ohm_inv_by_x_inv = np.linalg.inv(x_trans_by_ohm_inv_by_x)


Q1=x_trans_by_ohm_inv_by_x_inv
Q2=x_trans_by_ohm_inv_by_y

Q3=Q1.dot(Q1)

weight_gls = Q3
weight_gls=pd.DataFrame(data=weight_gls)

ET1=Q1
ET2 = Q2.to_numpy()
weights_for_gls=np.matmul(Q1,ET2)

weights_for_gls = pd.DataFrame(data=weights_for_gls)
#weights_for_gls



In [205]:
#FINDING THE TRUE Y FOR WLS
#Y=XB
ET3=new_x.to_numpy()
ET4=weights_for_gls.to_numpy()

true_y=np.matmul(ET3,ET4)
true_y = pd.DataFrame(data=true_y)
#true_y`

In [236]:
#FINDING THE RESIDUALS FOR WLS
residuals_for_gls=(y-true_y)**2

residuals_for_gls = pd.DataFrame(data=residuals_for_gls)
residuals_for_gls

residuals_for_gls=(np.subtract(y, true_y))**2
#residuals_for_gls

In [235]:
res=residuals_for_gls.to_numpy()
ols_resid = model.resid
res_fit = sm.OLS(list(res[1:]), list(res[:-1])).fit()
rho = res_fit.params
rho

ohm1=ohm.to_numpy()
from scipy.linalg import toeplitz
sigma = rho**ohm1

sigma.shape

ohm1=pd.DataFrame(data=ohm1)
#ohm1

In [249]:

##GLS Component 
ohm1=pd.DataFrame(data=ohm1)
ohm1.to_numpy()
ols_resid = model.resid

res_fit = sm.OLS(list(res[1:]), list(res[:-1])).fit()
#res_fit = sm.OLS(list(ols_resid[1:]), list(ols_resid[:-1])).fit()

from scipy.linalg import toeplitz

#toeplitz((residuals))
order = toeplitz(range(len(ols_resid)))
sigma = rho**order


sigma=pd.DataFrame(data=sigma)
gls_model = sm.GLS(y, new_x, sigma=sigma)
gls_results = gls_model.fit()
print(gls_results.summary())


                            GLS Regression Results                            
Dep. Variable:                   Texp   R-squared:                       0.775
Model:                            GLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     360.8
Date:                Fri, 16 Apr 2021   Prob (F-statistic):               0.00
Time:                        15:13:17   Log-Likelihood:                 1767.1
No. Observations:                1272   AIC:                            -3508.
Df Residuals:                    1259   BIC:                            -3441.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0751      0.014     79.275      0.0